In [1]:
import numpy as np
import pandas as pd
from statsmodels.formula.api import ols
from statsmodels.discrete.discrete_model import Logit
from sklearn.metrics import roc_auc_score

In [2]:
# LDL, Height, BMI, diabetes, renal failure, heart attack, asthma, non-melanoma skin cancer
pheno_list = ['INI50','INI21001','HC221','HC294','HC326','HC382','cancer1060']
phenos = pd.read_table('/oak/stanford/groups/mrivas/ukbb24983/phenotypedata/master_phe/master.phe',
                       usecols=['IID','age','sex','PC1','PC2','PC3','PC4']+pheno_list, 
                       index_col=['IID'],
                       na_values=-9)
phenos[[i for i in phenos.columns if 'INI' not in i]] -= 1
phenos.loc[~phenos.index.duplicated(keep='first')]
phenos.describe()

,age,sex,PC1,PC2,PC3,PC4,HC294,INI50,cancer1060,HC326,HC382,HC221,INI21001
count,488377.000000,488377.000000,488377.000000,488377.000000,488377.000000,488377.000000,488374.000000,500060.000000,488374.000000,488374.000000,488374.000000,488374.000000,499520.000000
mean,64.454925,-0.542335,-2.123313,-0.640132,-1.089341,-0.870598,0.021469,168.419126,0.046743,0.036075,0.129935,0.065823,27.433331
std,8.112788,0.498205,54.305102,27.858077,14.891779,10.403127,0.144942,9.276846,0.211088,0.186477,0.336233,0.247972,4.801300
min,45.000000,-1.000000,-20.269900,-283.317000,-144.258000,-108.573000,0.000000,75.000000,0.000000,0.000000,0.000000,0.000000,12.121200
25%,58.000000,-1.000000,-14.335200,1.445080,-3.673770,-1.910279,0.000000,161.500000,0.000000,0.000000,0.000000,0.000000,24.141500
50%,66.000000,-1.000000,-13.161900,2.648770,-2.506000,0.210960,0.000000,168.000000,0.000000,0.000000,0.000000,0.000000,26.743300
75%,71.000000,0.000000,-11.773500,3.762990,-1.248864,2.575510,0.000000,175.000000,0.000000,0.000000,0.000000,0.000000,29.904100
max,82.000000,0.000000,418.396000,85.112400,97.822600,36.103100,1.000000,209.000000,1.000000,1.000000,1.000000,1.000000,74.683700


In [3]:
a,b = 'white_british', 'non_british_white'
with open('/oak/stanford/groups/mrivas/ukbb24983/sqc/population_stratification/ukb24983_'+a+'.phe', 'r') as f:
    pop = {int(line.split()[0]) for line in f}

with open('/oak/stanford/groups/mrivas/ukbb24983/sqc/population_stratification/ukb24983_'+b+'.phe', 'r') as f:
    pop2 = {int(line.split()[0]) for line in f}

In [4]:
z='all_beta_center_p001_20190530_500PCs'
npz = np.load('/oak/stanford/groups/mrivas/projects/degas-risk/datasets/all_pop/'+z+'.npz')
scores = pd.read_table('/oak/stanford/groups/mrivas/projects/degas-risk/scorefiles/'+z+'_full.profile',
                       sep='\s+',
                       index_col='IID')
scores = scores.loc[scores.index.isin(phenos.index),:]
phenos = phenos.loc[phenos.index.isin(scores.index),:]

scores.iloc[:,:10].head()

,SCORE_PC1,SCORE_PC2,SCORE_PC3,SCORE_PC4,SCORE_PC5,SCORE_PC6,SCORE_PC7,SCORE_PC8,SCORE_PC9,SCORE_PC10
IID,,,,,,,,,,
-1,103.7650,-56.84110,-329.4330,72.4660,-226.1600,43.4152,77.91040,7.44528,-39.1203,-52.61950
-2,-936.5990,99.50940,-47.4708,-63.9602,-111.8800,-53.3234,-3.12852,-38.45870,114.5300,289.30500
-3,-330.6880,175.75900,201.2250,177.9940,21.2985,-16.4617,88.90270,58.59450,-54.6371,-2.89849
-4,-188.0230,58.26510,-250.1530,-23.0189,-110.5550,57.3824,33.25550,-51.58510,131.0040,176.63200
-5,24.3653,-7.05218,60.2319,74.4917,-301.1660,142.0400,-56.66530,-18.17940,61.7944,159.03100


In [5]:
for p,name in zip([pop,pop2],['White British:','Non-British White:']):
    print(name)
    for phe in pheno_list:
        print(phe),
        pcvec = npz['V'][np.where(npz['label_phe_code'] == phe),:].flatten()
        score = scores.loc[scores.index.isin(p),:].sort_index().dot(pcvec).dropna()
        if 'INI' in phe:
            pheno = ols(phe+'~age+sex+PC1+PC2+PC3+PC4',
                        data=phenos.loc[phenos.index.isin(p),:]).fit().resid.sort_index().dropna()
            pheno = pheno.subtract(pheno.mean()).divide(pheno.std())
        else:
            pheno = phenos.loc[phenos.index.isin(p),phe].sort_index().dropna()
        data = pd.concat([score,pheno], axis=1)
        if 'INI' in phe:
            print('r^2={:.4}'.format(data.corr(method='spearman').iloc[0,1]**2))
        else:
            print('AUC={:.4}'.format(roc_auc_score(data.iloc[:,1].astype(bool), data.iloc[:,0])))
    print('')

White British:
INI50 r^2=0.2129
INI21001 r^2=0.1402
HC221 AUC=0.7357
HC294 AUC=0.6774
HC326 AUC=0.8682
HC382 AUC=0.6392
cancer1060 AUC=0.7659

Non-British White:
INI50 r^2=0.1732
INI21001 r^2=0.05086
HC221 AUC=0.5895
HC294 AUC=0.5432
HC326 AUC=0.5759
HC382 AUC=0.5726
cancer1060 AUC=0.6012

